<a href="https://colab.research.google.com/github/rdmurugan/Orderbook-Code/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import base64
import requests
import json
import pandas as pd
from pandas.core.reshape.concat import concat
from pandas.core.frame import DataFrame

def covert_ob_to_dataframe_binance(obdata: dict, exchange, symbol) -> DataFrame:
    obframes = {side: pd.DataFrame(data=obdata[side], columns=['price', 'quantity'], dtype=float) for side in ["bids", "asks"]}
    obframes_list = [obframes[side].assign(side=side) for side in obframes] 
    _obdata = pd.concat(obframes_list, axis='index', ignore_index=True, sort=True)
    #range_series = _obdata["price"].between(MIN_VALUE, MAX_VALUE, inclusive = True)
    _obdata ['cost']  = _obdata ['price'] * _obdata ['quantity']
    _obdata['datatimeload'] = pd.to_datetime('now').strftime("%Y-%m-%d %H:%M")
    _obdata['exchange'] = exchange
    _obdata['symbol'] = symbol
    return _obdata


def covert_ob_to_dataframe_coinbase(obdata: dict, exchange, symbol) -> DataFrame:
    obframes = {side: pd.DataFrame(data=obdata[side], columns=['price', 'quantity', 'size'], dtype=float) for side in ["bids", "asks"]}
    obframes_list = [obframes[side].assign(side=side) for side in obframes] 
    _obdata = pd.concat(obframes_list, axis="index", ignore_index=True, sort=True)
    _obdata = _obdata.drop('size', 1)
    #range_series = _obdata["price"].between(MIN_VALUE, MAX_VALUE, inclusive = True)
    #_obdata  =_obdata[range_series]
    _obdata ['cost']  = _obdata ['price'] * _obdata ['quantity']
    _obdata['datatimeload'] = pd.to_datetime('now').strftime("%Y-%m-%d %H:%M")
    _obdata['exchange'] = exchange
    _obdata['symbol'] = symbol
    return _obdata

project_id = "coredata-trial"
topic_id = "Orderbookfromexchanges"
    
    # get some orderbook data

binance_btc_api = "https://www.binance.com/api/v3/depth?symbol=BTCBUSD"
binance_eth_api = "https://www.binance.com/api/v3/depth?symbol=ETHBUSD"
coinbase_btc_api="https://api.pro.coinbase.com/products/BTC-USD/book?level=2"
coinbase_eth_api="https://api.pro.coinbase.com/products/ETH-USD/book?level=2"

obreq_binance_btc =  requests.get(binance_btc_api).json();
obreq_binance_eth =  requests.get(binance_eth_api).json();
obreq_coinbase_btc =  requests.get(coinbase_btc_api).json();
obreq_coinbase_eth =  requests.get(coinbase_eth_api).json();

pd_bin_btc = covert_ob_to_dataframe_binance(obreq_binance_btc,"Binance", "BTC")
pd_bin_eth = covert_ob_to_dataframe_binance(obreq_binance_eth,"Binance", "ETH")
pd_cb_btc = covert_ob_to_dataframe_coinbase(obreq_coinbase_btc,"Coinbase", "BTC")
pd_cb_eth = covert_ob_to_dataframe_coinbase(obreq_coinbase_eth,"Coinbase", "ETH")

obframes_pd = [pd_bin_btc, pd_bin_eth, pd_cb_btc, pd_cb_eth ] 
_obdata_list = pd.concat(obframes_pd)

ob_dictionary = [ dict([(colname, row[i])  for i,colname in enumerate(_obdata_list.columns)  ])  for row in _obdata_list.values ]  
ob_message = json.dumps(ob_dictionary).encode("utf-8")
print(ob_message)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
ob_anlayse = _obdata_list[_obdata_list.cost >= 50000]
perc =[.20, .40, .60, .80]

In [ ]:
ob_anlayse.describe(percentiles = perc)
  

,price,quantity,cost
count,3.371000e+03,3371.000000,3.371000e+03
mean,2.068206e+07,146.930458,8.832230e+06
std,3.724099e+08,2812.789048,3.876459e+08
min,1.000000e+00,0.000170,5.000000e+04
20%,4.912020e+03,1.023149,7.197800e+04
40%,2.479200e+04,3.676242,1.171439e+05
50%,3.399000e+04,6.963081,1.584345e+05
60%,4.998800e+04,14.014398,2.346976e+05
80%,1.088889e+05,57.013415,8.026600e+05
max,1.000000e+10,143371.592731,2.240581e+10


In [ ]:
from google.cloud import storage
def store_data_in_bucket(df: bytes):
    # Instantiates a client
    project_id = "coredata-trial"
    client = storage.Client(project_id)
    # Creates a new bucket and uploads an object
    fname = "orderbook" + pd.to_datetime('now').strftime("%Y-%m-%d-%H-%M")+".json"
    bucket = client.bucket("coredatastore001")
    blob = bucket.blob(fname)
    blob.upload_from_string(
          data=ob_message,
          content_type='application/json'
        )    
    print(f"Wrote json with pandas with name {blob.name} to the bucket {bucket.name}.")

In [ ]:
!gcloud config set project "coredata-trial"

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [ ]:
from google.colab import auth
auth.authenticate_user()
project_id = "coredata-trial"


store_data_in_bucket(ob_message)

Wrote json with pandas with name orderbook2022-02-17-02-44.json to the bucket coredatastore001.


In [ ]:
project_id = "coredata-trial"
client = storage.Client(project="coredata-trial")
# Creates a new bucket and uploads an object
fname = "orderbook" + pd.to_datetime('now').strftime("%Y-%m-%d-%H-%M")+".json"
bucket = client.bucket("coredatastore001")
 

In [ ]:
blob = bucket.blob(fname)
print(blob)




<Blob: coredatastore001, orderbook2022-02-16-21-02.json, None>


In [ ]:
with blob.open("w") as f:
   f.write(df.to_json(orient='table'))
print(f"Wrote json with pandas with name {blob_name} from bucket {bucket.name}.")

AttributeError: ignored

In [ ]:


import base64
import requests
import json
import pandas as pd
from pandas.core.reshape.concat import concat
from pandas.core.frame import DataFrame

binance_btc_api = "https://www.binance.com/api/v3/depth?symbol=BTCBUSD&limit=1000"
obreq_binance_btc =  requests.get(binance_btc_api).json();





obframes = {side: pd.DataFrame(data=obreq_binance_btc[side], columns=['price', 'quantity'], dtype=float) for side in ["bids", "asks"]}
obframes_list = [obframes[side].assign(side=side) for side in obframes] 
_obdata = pd.concat(obframes_list, axis='index', ignore_index=True, sort=True)

MIN_VALUE = 30000
MAX_VALUE = 100000

# making a bool series
bool_series = _obdata["price"].between(MIN_VALUE, MAX_VALUE, inclusive = True)
  
# returning dataframe with salary between above value
_obdata  =_obdata[bool_series]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.


In [ ]:
print(obreq_binance_btc)

{'lastUpdateId': 8377419708, 'bids': [['40580.11000000', '0.04530000'], ['40579.02000000', '0.00582000'], ['40578.94000000', '0.01376000'], ['40578.35000000', '0.02736000'], ['40577.84000000', '0.00231000'], ['40577.03000000', '0.00040000'], ['40576.41000000', '0.00050000'], ['40575.25000000', '0.00050000'], ['40574.98000000', '0.07609000'], ['40574.97000000', '0.06000000'], ['40574.68000000', '0.12974000'], ['40573.88000000', '0.04962000'], ['40573.87000000', '0.12214000'], ['40573.80000000', '0.00493000'], ['40573.67000000', '0.00119000'], ['40572.62000000', '0.07940000'], ['40572.47000000', '0.09774000'], ['40572.46000000', '0.04062000'], ['40571.85000000', '0.05000000'], ['40571.17000000', '1.04910000'], ['40571.16000000', '2.41251000'], ['40570.78000000', '0.01022000'], ['40570.31000000', '0.12323000'], ['40569.40000000', '0.00511000'], ['40568.92000000', '0.19338000'], ['40568.51000000', '0.00450000'], ['40567.78000000', '0.04141000'], ['40566.06000000', '0.05000000'], ['40565.37

In [ ]:
thisdict = {
  'brand':[['1.0','2.0'], ['3.0','4.0']],
  'model':[['1.1','2.1'],['3.1','4.1' ]],
  'year': '1964'
 }

obframe={side:pd.DataFrame (data = thisdict[side] , columns =['brand', 'model'] ) for side in ['brand','model']}
ob_list = [obframe[side].assign(side=side) for side in obframe] 

In [ ]:
print(ob_list)

[  brand model   side
0   1.0   2.0  brand
1   3.0   4.0  brand,   brand model   side
0   1.1   2.1  model
1   3.1   4.1  model]


In [ ]:
v = lambda a : a + 10 

In [ ]:
print(v(10))

20


In [ ]:
def myfunc(n) :
  return lambda a : a * n
mydoubler = myfunc(2)
mytripler = myfunc(3)

print(mydoubler(11))
print(mytripler(11))

22
33
